In [3]:
# Import necessary modules
from flask import Flask, request, jsonify
from dotenv import load_dotenv
import openai
from openai import OpenAI
import uuid
import os

In [4]:
load_dotenv()
OPEN_AI_API_KEY = os.getenv("API_KEY")
client = OpenAI(api_key = OPEN_AI_API_KEY)

In [2]:
app = Flask(__name__)

In [5]:
documents = {}
test_answers = {}

In [ ]:
#Endpoint to upload research paper
@app.route("/upload", methods=['POST'])
def upload():
    if "files" not in request.files:                  #check if file is empty
        return jsonify({"Error": "No File "}), 400
    file = request.files['files']
    document_id = str(uuid.uuid4())

    #storing document
    documents[document_id] = file.read().decode("utf-8")
    test_answer = Gen_answer(documents[document_id])
    test_answers[document_id] = test_answer              #tricky variable declaration
    return jsonify({"document_id": document_id, "message":"Document Successfully uploaded"}), 200

#utilizing Openai APi to generate test answer
def Gen_answer(documents):    
    response = assistant.create(
        model="gpt-3.5-turbo-1106",
        messages=[
            {"role": "system", "content": "You are my research assistant."},
            {"role": "user", "content": f"Generate a test answer for the following document:\n{document_text}"}
        ]
    )
    return response['choices'][0]['message']['content']

In [ ]:
#Endpoint to many the Q>A>T session
@app.route('/query/<string:document_id>/', methods=['POST'])
def query_document(document_id):
    if document_id not in documents:
        return jsonify({"error": "Document not found"}), 404

    data = request.get_json()
    question = data.get("question")

    if not question:
        return jsonify({"error": "Question not provided"}), 400
    
    #Generate test Question
    test_questions_id = str(uuid.uuid4)
    test_question = generate_test_question(question)

    # Simulate a test answer retrieval from the database
    #test_answer = test_answers.get(document_id, "") 

    response = {
        "answer": generate_answer(question, documents[document_id]),
        "bullet_points": ["Point 1", "Point 2", "Point 3"],
        "test_question": test_question,
        "test_question_id": test_questions_id,
    }

    return jsonify(response)

# Generate a test question using OpenAI Assistant API
def generate_test_question(question):
    # Use OpenAI Assistant API to generate a test question based on the user's question
    response = assistant.append_message(
        model="text-davinci-003",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": f"Generate a test question for the following user question:\n{question}"}
        ]
    )
    
    test_question_content = response['choices'][0]['message']['content']
    
    return test_question_content

# Generate an answer using OpenAI Assistant API
def generate_answer(question, document_text):
    # Use OpenAI Assistant API to generate an answer based on the user's question and document
    response = assistant.append_message(
        model="text-davinci-003",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": f"{question}\nDocument: {document_text}"}
        ]
    )
    
    answer_content = response['choices'][0]['message']['content']
    
    return answer_content

In [ ]:
# Endpoint for evaluating user response to the test question
@app.route('/evaluate/<string:test_question_id>/', methods=['POST'])
def evaluate_response(test_question_id):
    data = request.get_json()
    user_answer = data.get("user_answer")

    if not user_answer:
        return jsonify({"error": "User answer not provided"}), 400

    # Simulate fetching the correct test answer from the database
    correct_answer = test_answers.get(test_question_id, "")

    # Compare user's answer with the correct answer
    knowledge_understood = (user_answer.lower() == correct_answer.lower())
    
    # Confidence is set to 75% for simplicity
    knowledge_confidence = 75

    response = {
        "knowledge_understood": knowledge_understood,
        "knowledge_confidence": knowledge_confidence
    }

    return jsonify(response)


In [ ]:
if __name__ == "__main__":
    app.run()